In [1524]:
import pandas as pd
# Remove the limit on the number of columns displayed
pd.set_option('display.max_columns', None)

In [1525]:
table_df = pd.read_csv("finaltable1.csv")

In [1526]:
table_df.head()

,Unnamed: 0.1,Match ID,Home Team,Away Team,Home Score,Away Score,Winner,Prev Home Played,Prev Home Won,Prev Home Drawn,Prev Home Lost,Prev Home Goals Scored,Prev Home Goals Conceded,Prev Home Goal Difference,Prev Home Points,Prev Home Form,Prev Away Played,Prev Away Won,Prev Away Drawn,Prev Away Lost,Prev Away Goals Scored,Prev Away Goals Conceded,Prev Away Goal Difference,Prev Away Points,Prev Away Form,Date,Time,Wk,Day,xG,Score,xG.1,Attendance,Venue,Referee,Away Win Odds,Home Win Odds,Draw Odds,StadiumID,Latitude,Longitude,Datetime,Temperature,Precipitation,H2H_Home_Wins_Last_4,H2H_Away_Wins_Last_4,H2H_Draws_Last_4,Composite Key,away_team_code,home_team_code,home_xg,away_xg,is_result,has_data,url,away_points,away_expected_points,away_np_xg,away_np_xg_difference,away_ppda,away_deep_completions,home_points,home_expected_points,home_np_xg,home_np_xg_difference,home_ppda,home_deep_completions,Unnamed: 0,gameweek,home_team_fdr,away_team_fdr,home_team,away_team,composite_key,HomeTeam,AwayTeam,B365H,B365D,B365A
0,0,497410,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-16,19:00,1,Fri,2.4,1–0,0.4,73297.0,Old Trafford,Robert Jones,NaN,NaN,NaN,1.0,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,0.0,3,1,0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1,1.0,2,3,Manchester Utd,Fulham,2024-08-16_Manchester Utd_Fulham,Manchester Utd,Fulham,1.60,4.20,5.25
1,1,497411,Ipswich Town,Liverpool,0.0,2.0,AWAY_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,11:30,1,Sat,0.5,0–2,2.6,30014.0,Portman Road Stadium,Tim Robinson,NaN,NaN,NaN,2.0,52.0567,1.1450,2024-08-17T11:30:00,22.212500,0.0,0,0,0,2024-08-17_Ipswich Town_Liverpool,LIV,IPS,0.342601,3.929060,True,True,https://understat.com/match/26603,3.0,2.9562,3.929060,3.586459,8.739130,13.0,0.0,0.0258,0.342601,-3.586459,18.777778,2.0,2,1.0,5,2,Ipswich Town,Liverpool,2024-08-17_Ipswich Town_Liverpool,Ipswich Town,Liverpool,8.50,5.50,1.33
2,2,497412,Arsenal,Wolves,2.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,1.2,2–0,0.5,60261.0,Emirates Stadium,Jarred Gillett,NaN,NaN,NaN,3.0,51.5550,-0.1086,2024-08-17T14:00:00,21.734999,0.0,4,0,0,2024-08-17_Arsenal_Wolves,WOL,ARS,1.628300,0.575835,True,True,https://understat.com/match/26604,0.0,0.5796,0.575835,-1.052465,10.818182,2.0,3.0,2.1894,1.628300,1.052465,7.769231,14.0,3,1.0,2,5,Arsenal,Wolves,2024-08-17_Arsenal_Wolves,Arsenal,Wolves,1.18,7.50,13.00
3,3,497413,Everton,Brighton,0.0,3.0,AWAY_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,0.5,0–3,1.4,39217.0,Goodison Park,Simon Hooper,NaN,NaN,NaN,4.0,53.4389,-2.9664,2024-08-17T14:00:00,17.587500,0.0,0,0,0,2024-08-17_Everton_Brighton,BRI,EVE,0.405325,1.790830,True,True,https://understat.com/match/26605,3.0,2.5287,1.790830,1.385505,7.916667,5.0,0.0,0.3078,0.405325,-1.385505,18.333333,4.0,4,1.0,2,2,Everton,Brighton,2024-08-17_Everton_Brighton,Everton,Brighton,2.63,3.30,2.63
4,4,497414,Newcastle Utd,Southampton,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,0.3,1–0,1.8,52196.0,St James' Park,Craig Pawson,NaN,NaN,NaN,5.0,54.9756,-1.6217,2024-08-17T14:00:00,18.622499,0.0,0,0,0,2024-08-17_Newcastle Utd_Southampton,SOU,NEW,0.433489,1.954830,True,True,https://understat.com/match/26606,0.0,2.5586,1.954830,1.521341,3.789474,13.0,3.0,0.2774,0.433489,-1.521341,16.250000,4.0,5,1.0,2,4,Newcastle Utd,Southampton,2024-08-17_Newcastle Utd_Southampton,Newcastle Utd,Southampton,1.36,5.25,8.00


In [1527]:
len(table_df)

369

# Data Preprocessing

## Handling Missing Values

In [1530]:
def check_missing_values():
    # Check for missing values in the dataset
    missing_values = table_df.isnull().sum()
    
    # Filter for columns with missing values
    missing_values = missing_values[missing_values > 0]
    
    # Display missing values
    print("Columns with Missing Values:")
    print(missing_values)


In [1531]:
check_missing_values()

Columns with Missing Values:
Home Score               220
Away Score               220
Winner                   220
xG                       220
Score                    220
xG.1                     220
Attendance               220
Referee                  220
Away Win Odds            360
Home Win Odds            360
Draw Odds                360
Temperature              239
Precipitation            239
away_team_code            24
home_team_code            24
home_xg                  220
away_xg                  220
is_result                 24
has_data                  24
url                       24
away_points              220
away_expected_points     220
away_np_xg               220
away_np_xg_difference    220
away_ppda                220
away_deep_completions    220
home_points              220
home_expected_points     220
home_np_xg               220
home_np_xg_difference    220
home_ppda                220
home_deep_completions    220
HomeTeam                 220
AwayTeam      

Remove matches that have not been played..

In [1533]:
table_df = table_df.dropna(subset=['Home Score', 'Away Score'])
# Reset the index after filtering
table_df.reset_index(drop=True, inplace=True)

In [1534]:
check_missing_values()

Columns with Missing Values:
Away Win Odds    149
Home Win Odds    149
Draw Odds        149
Temperature       19
Precipitation     19
dtype: int64


In [1535]:
# Fill missing 'xG' values with corresponding 'home_xg' values for the same row
table_df['xG'] = table_df.apply(
    lambda row: row['home_xg'] if pd.isnull(row['xG']) else row['xG'],
    axis=1
)
table_df['xG.1'] = table_df.apply(
    lambda row: row['away_xg'] if pd.isnull(row['xG.1']) else row['xG.1'],
    axis=1
)

# Verify that there are no more missing values in 'xG'
missing_xg_after = table_df['xG'].isnull().sum()
print(f"Remaining missing values in 'xG': {missing_xg_after}")
missing_xg_after = table_df['xG.1'].isnull().sum()
print(f"Remaining missing values in 'xG.1': {missing_xg_after}")


Remaining missing values in 'xG': 0
Remaining missing values in 'xG.1': 0


In [1536]:
check_missing_values()

Columns with Missing Values:
Away Win Odds    149
Home Win Odds    149
Draw Odds        149
Temperature       19
Precipitation     19
dtype: int64


Drop the score column because we already have home scores and away scores

In [1538]:
# Drop the 'Score' column from the dataset
table_df.drop(columns=['Score','Precipitation'], inplace=True)

In [1539]:
check_missing_values()

Columns with Missing Values:
Away Win Odds    149
Home Win Odds    149
Draw Odds        149
Temperature       19
dtype: int64


In [1540]:
# Fill missing 'Attendance' values with the median attendance
median_attendance = table_df['Attendance'].median()
table_df['Attendance'].fillna(median_attendance, inplace=True)

# Verify there are no missing values in 'Attendance'
print(f"Remaining missing values in 'Attendance': {table_df['Attendance'].isnull().sum()}")


Remaining missing values in 'Attendance': 0


In [1541]:
# Filter rows where 'Referee' is null
rows_with_missing_referee = table_df[table_df['Referee'].isnull()]
rows_with_missing_referee.head(7)

,Unnamed: 0.1,Match ID,Home Team,Away Team,Home Score,Away Score,Winner,Prev Home Played,Prev Home Won,Prev Home Drawn,Prev Home Lost,Prev Home Goals Scored,Prev Home Goals Conceded,Prev Home Goal Difference,Prev Home Points,Prev Home Form,Prev Away Played,Prev Away Won,Prev Away Drawn,Prev Away Lost,Prev Away Goals Scored,Prev Away Goals Conceded,Prev Away Goal Difference,Prev Away Points,Prev Away Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,Away Win Odds,Home Win Odds,Draw Odds,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Wins_Last_4,H2H_Away_Wins_Last_4,H2H_Draws_Last_4,Composite Key,away_team_code,home_team_code,home_xg,away_xg,is_result,has_data,url,away_points,away_expected_points,away_np_xg,away_np_xg_difference,away_ppda,away_deep_completions,home_points,home_expected_points,home_np_xg,home_np_xg_difference,home_ppda,home_deep_completions,Unnamed: 0,gameweek,home_team_fdr,away_team_fdr,home_team,away_team,composite_key,HomeTeam,AwayTeam,B365H,B365D,B365A


In [1542]:
# Fill missing 'Referee' values with the mode (most common referee)
mode_referee = table_df['Referee'].mode()[0]
table_df['Referee'].fillna(mode_referee, inplace=True)

# Verify there are no missing values in 'Referee'
print(f"Remaining missing values in 'Referee': {table_df['Referee'].isnull().sum()}")


Remaining missing values in 'Referee': 0


In [1543]:
check_missing_values()

Columns with Missing Values:
Away Win Odds    149
Home Win Odds    149
Draw Odds        149
Temperature       19
dtype: int64


In [1544]:
from sklearn.impute import KNNImputer
import pandas as pd

# Load the dataset (assuming it's in table_df)
# Replace 'Temperature' and any other relevant columns for KNN imputation
columns_to_impute = ['Temperature']  # Include columns correlated with Temperature

# Select only the relevant columns for imputation
impute_df = table_df[columns_to_impute]

# Initialize the KNN imputer
knn_imputer = KNNImputer(n_neighbors=5)  # Use 5 neighbors by default

# Perform KNN imputation
imputed_data = knn_imputer.fit_transform(impute_df)

# Replace the original 'Temperature' column with the imputed values
table_df['Temperature'] = imputed_data[:, 0]

# Verify the imputation
print("Remaining missing values in 'Temperature':", table_df['Temperature'].isnull().sum())


Remaining missing values in 'Temperature': 0


In [1545]:
check_missing_values()

Columns with Missing Values:
Away Win Odds    149
Home Win Odds    149
Draw Odds        149
dtype: int64


In [1546]:
columns_to_remove = ['Away Win Odds', 'Home Win Odds', 'Draw Odds']
table_df.drop(columns=columns_to_remove, inplace=True)


In [1547]:
check_missing_values()

Columns with Missing Values:
Series([], dtype: int64)


## Check for outliers

In [1549]:
# Function to identify outliers based on IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)  # First quartile
    Q3 = df[column].quantile(0.75)  # Third quartile
    IQR = Q3 - Q1  # Interquartile range
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

# Check for outliers in numerical columns
numerical_columns = table_df.select_dtypes(include=['float64', 'int64']).columns
outliers = {col: detect_outliers_iqr(table_df, col) for col in numerical_columns}

# Display the columns with outliers and their counts
outlier_counts = {col: len(outliers[col]) for col in outliers if len(outliers[col]) > 0}
print("Columns with outliers and their counts:")
print(outlier_counts)


Columns with outliers and their counts:
{'Home Score': 13, 'Away Score': 7, 'Prev Home Lost': 2, 'Prev Home Goal Difference': 3, 'Prev Away Won': 2, 'Prev Away Lost': 3, 'Prev Away Goals Conceded': 2, 'Prev Away Goal Difference': 6, 'Prev Away Points': 1, 'xG': 4, 'xG.1': 9, 'Temperature': 9, 'H2H_Home_Wins_Last_4': 5, 'H2H_Away_Wins_Last_4': 5, 'H2H_Draws_Last_4': 2, 'home_xg': 2, 'away_xg': 4, 'away_np_xg': 3, 'away_np_xg_difference': 4, 'away_ppda': 7, 'away_deep_completions': 6, 'home_np_xg': 2, 'home_np_xg_difference': 4, 'home_ppda': 8, 'home_deep_completions': 4, 'B365H': 13, 'B365D': 11, 'B365A': 9}


### Column names changed for feature engineering

In [1551]:
import pandas as pd

data = table_df
# Mapping the old column names to new names where 'Home' and 'Away' are replaced with 'Team 1' and 'Team 2'
column_mapping = {
    'Home Team': 'Team 1',
    'Away Team': 'Team 2',
    'Home Score': 'Team 1 Score',
    'Away Score': 'Team 2 Score',
    'Prev Home Played': 'Prev Team 1 Played',
    'Prev Away Played': 'Prev Team 2 Played',
    'Prev Home Won': 'Prev Team 1 Won',
    'Prev Home Drawn': 'Prev Team 1 Drawn',
    'Prev Home Lost': 'Prev Team 1 Lost',
    'Prev Home Goals Scored': 'Prev Team 1 Goals Scored',
    'Prev Home Goals Conceded': 'Prev Team 1 Goals Conceded',
    'Prev Home Goal Difference': 'Prev Team 1 Goal Difference',
    'Prev Home Points': 'Prev Team 1 Points',
    'Prev Home Form': 'Prev Team 1 Form',
    'Prev Away Won': 'Prev Team 2 Won',
    'Prev Away Drawn': 'Prev Team 2 Drawn',
    'Prev Away Lost': 'Prev Team 2 Lost',
    'Prev Away Goals Scored': 'Prev Team 2 Goals Scored',
    'Prev Away Goals Conceded': 'Prev Team 2 Goals Conceded',
    'Prev Away Goal Difference': 'Prev Team 2 Goal Difference',
    'Prev Away Points': 'Prev Team 2 Points',
    'Prev Away Form': 'Prev Team 2 Form',
    'H2H_Home_Wins_Last_4': 'H2H_Team_1_Wins_Last_4',
    'H2H_Away_Wins_Last_4': 'H2H_Team_2_Wins_Last_4',
    'H2H_Draws_Last_4': 'H2H_Draws_Last_4',
    'home_team_code': 'team_1_code',
    'away_team_code': 'team_2_code',
    'home_xg': 'team_1_xg',
    'away_xg': 'team_2_xg',
    'home_team': 'team_1',
    'away_team': 'team_2',
    'HomeTeam': 'Team1',
    'AwayTeam': 'Team2',
    'home_team_fdr': 'team_1_fdr',
    'away_team_fdr': 'team_2_fdr',
    'home_deep_completions': 'team1_deep_completions',
    'away_deep_completions': 'team2_deep_completions',
    'away_np_xg': 'team2_np_xg',
    'home_np_xg': 'team1_np_xg',
    'away_np_xg_difference':'team2_np_xg_difference',
    'home_np_xg_difference':'home_np_xg_difference',
    'away_np_xg': 'team2_np_xg',
    'home_ppda':'team1_ppda', 
    'away_ppda': 'team2_ppda',
    'home_points': 'team1_points',
    'away_points': 'team2_points',
    'away_expected_points': 'team2_expected_points',
    'home_expected_points':'team1_expected_points',
}

# Renaming the columns in the dataframe
data.rename(columns=column_mapping, inplace=True)
data.columns


Index(['Unnamed: 0.1', 'Match ID', 'Team 1', 'Team 2', 'Team 1 Score',
       'Team 2 Score', 'Winner', 'Prev Team 1 Played', 'Prev Team 1 Won',
       'Prev Team 1 Drawn', 'Prev Team 1 Lost', 'Prev Team 1 Goals Scored',
       'Prev Team 1 Goals Conceded', 'Prev Team 1 Goal Difference',
       'Prev Team 1 Points', 'Prev Team 1 Form', 'Prev Team 2 Played',
       'Prev Team 2 Won', 'Prev Team 2 Drawn', 'Prev Team 2 Lost',
       'Prev Team 2 Goals Scored', 'Prev Team 2 Goals Conceded',
       'Prev Team 2 Goal Difference', 'Prev Team 2 Points', 'Prev Team 2 Form',
       'Date', 'Time', 'Wk', 'Day', 'xG', 'xG.1', 'Attendance', 'Venue',
       'Referee', 'StadiumID', 'Latitude', 'Longitude', 'Datetime',
       'Temperature', 'H2H_Team_1_Wins_Last_4', 'H2H_Team_2_Wins_Last_4',
       'H2H_Draws_Last_4', 'Composite Key', 'team_2_code', 'team_1_code',
       'team_1_xg', 'team_2_xg', 'is_result', 'has_data', 'url',
       'team2_points', 'team2_expected_points', 'team2_np_xg',
       'tea

In [1552]:
data.head()

,Unnamed: 0.1,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Team_1_Wins_Last_4,H2H_Team_2_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,home_np_xg_difference,team1_ppda,team1_deep_completions,Unnamed: 0,gameweek,team_1_fdr,team_2_fdr,team_1,team_2,composite_key,Team1,Team2,B365H,B365D,B365A
0,0,497410,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-16,19:00,1,Fri,2.4,0.4,73297.0,Old Trafford,Robert Jones,1.0,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,3,1,0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1,1.0,2,3,Manchester Utd,Fulham,2024-08-16_Manchester Utd_Fulham,Manchester Utd,Fulham,1.60,4.20,5.25
1,1,497411,Ipswich Town,Liverpool,0.0,2.0,AWAY_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,11:30,1,Sat,0.5,2.6,30014.0,Portman Road Stadium,Tim Robinson,2.0,52.0567,1.1450,2024-08-17T11:30:00,22.212500,0,0,0,2024-08-17_Ipswich Town_Liverpool,LIV,IPS,0.342601,3.929060,True,True,https://understat.com/match/26603,3.0,2.9562,3.929060,3.586459,8.739130,13.0,0.0,0.0258,0.342601,-3.586459,18.777778,2.0,2,1.0,5,2,Ipswich Town,Liverpool,2024-08-17_Ipswich Town_Liverpool,Ipswich Town,Liverpool,8.50,5.50,1.33
2,2,497412,Arsenal,Wolves,2.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,1.2,0.5,60261.0,Emirates Stadium,Jarred Gillett,3.0,51.5550,-0.1086,2024-08-17T14:00:00,21.734999,4,0,0,2024-08-17_Arsenal_Wolves,WOL,ARS,1.628300,0.575835,True,True,https://understat.com/match/26604,0.0,0.5796,0.575835,-1.052465,10.818182,2.0,3.0,2.1894,1.628300,1.052465,7.769231,14.0,3,1.0,2,5,Arsenal,Wolves,2024-08-17_Arsenal_Wolves,Arsenal,Wolves,1.18,7.50,13.00
3,3,497413,Everton,Brighton,0.0,3.0,AWAY_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,0.5,1.4,39217.0,Goodison Park,Simon Hooper,4.0,53.4389,-2.9664,2024-08-17T14:00:00,17.587500,0,0,0,2024-08-17_Everton_Brighton,BRI,EVE,0.405325,1.790830,True,True,https://understat.com/match/26605,3.0,2.5287,1.790830,1.385505,7.916667,5.0,0.0,0.3078,0.405325,-1.385505,18.333333,4.0,4,1.0,2,2,Everton,Brighton,2024-08-17_Everton_Brighton,Everton,Brighton,2.63,3.30,2.63
4,4,497414,Newcastle Utd,Southampton,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,0.3,1.8,52196.0,St James' Park,Craig Pawson,5.0,54.9756,-1.6217,2024-08-17T14:00:00,18.622499,0,0,0,2024-08-17_Newcastle Utd_Southampton,SOU,NEW,0.433489,1.954830,True,True,https://understat.com/match/26606,0.0,2.5586,1.954830,1.521341,3.789474,13.0,3.0,0.2774,0.433489,-1.521341,16.250000,4.0,5,1.0,2,4,Newcastle Utd,Southampton,2024-08-17_Newcastle Utd_Southampton,Newcastle Utd,Southampton,1.36,5.25,8.00


### FE: Average fdr

In [1553]:
def calculate_fdr_cumulative_features(dataframe):
    # Initialize dictionaries to track cumulative sums and games played
    cumulative_fdr = {}
    games_played = {}
    
    # Create new columns for average FDR and cumulative FDR
    avg_team_1_fdr = []
    avg_team_2_fdr = []
    cum_team_1_fdr = []
    cum_team_2_fdr = []

    # Iterate over rows
    for index, row in dataframe.iterrows():
        team_1 = row["Team 1"]
        team_2 = row["Team 2"]
        
        # Initialize if not present in dictionary
        if team_1 not in cumulative_fdr:
            cumulative_fdr[team_1] = 0
            games_played[team_1] = 0
        if team_2 not in cumulative_fdr:
            cumulative_fdr[team_2] = 0
            games_played[team_2] = 0

        # Update cumulative FDR
        cumulative_fdr[team_1] += row['team_1_fdr']
        cumulative_fdr[team_2] += row['team_2_fdr']
        
        # Increment games played
        games_played[team_1] += 1
        games_played[team_2] += 1

         # Record cumulative FDR for the current row
        cum_team_1_fdr.append(cumulative_fdr[team_1])
        cum_team_2_fdr.append(cumulative_fdr[team_2])
        
        # Calculate averages for teams before updating current match FDR
       
        avg_team_1_fdr.append(cumulative_fdr[team_1] / (games_played[team_1])) 
        avg_team_2_fdr.append(cumulative_fdr[team_2] / (games_played[team_2]))

    # Add new columns to the DataFrame
    dataframe['Cum Team 1 FDR'] = cum_team_1_fdr
    dataframe['Cum Team 2 FDR'] = cum_team_2_fdr
    dataframe['Avg Team 1 FDR'] = avg_team_1_fdr
    dataframe['Avg Team 2 FDR'] = avg_team_2_fdr

    

    return dataframe

# Calculate cumulative features excluding current match
detailed_fixtures = calculate_fdr_cumulative_features(data)

# Display selected rows with cumulative values and averages
columns_to_display = [
    'Match ID', 'Team 1', 'Team 2', 'team_1_fdr', 'team_2_fdr',
    'Cum Team 1 FDR', 'Cum Team 2 FDR', 'Avg Team 1 FDR', 'Avg Team 2 FDR'
]



In [1554]:
detailed_fixtures[['Team 1','Team 2','Prev Team 1 Played',
'Prev Team 2 Played','Avg Team 1 FDR','Avg Team 2 FDR','team_1_fdr', 'team_2_fdr','Cum Team 1 FDR','Cum Team 2 FDR']].to_csv("detailed_fixture.csv")

In [1555]:
table_df.head()

,Unnamed: 0.1,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Team_1_Wins_Last_4,H2H_Team_2_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,home_np_xg_difference,team1_ppda,team1_deep_completions,Unnamed: 0,gameweek,team_1_fdr,team_2_fdr,team_1,team_2,composite_key,Team1,Team2,B365H,B365D,B365A,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR
0,0,497410,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-16,19:00,1,Fri,2.4,0.4,73297.0,Old Trafford,Robert Jones,1.0,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,3,1,0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1,1.0,2,3,Manchester Utd,Fulham,2024-08-16_Manchester Utd_Fulham,Manchester Utd,Fulham,1.60,4.20,5.25,2,3,2.0,3.0
1,1,497411,Ipswich Town,Liverpool,0.0,2.0,AWAY_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,11:30,1,Sat,0.5,2.6,30014.0,Portman Road Stadium,Tim Robinson,2.0,52.0567,1.1450,2024-08-17T11:30:00,22.212500,0,0,0,2024-08-17_Ipswich Town_Liverpool,LIV,IPS,0.342601,3.929060,True,True,https://understat.com/match/26603,3.0,2.9562,3.929060,3.586459,8.739130,13.0,0.0,0.0258,0.342601,-3.586459,18.777778,2.0,2,1.0,5,2,Ipswich Town,Liverpool,2024-08-17_Ipswich Town_Liverpool,Ipswich Town,Liverpool,8.50,5.50,1.33,5,2,5.0,2.0
2,2,497412,Arsenal,Wolves,2.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,1.2,0.5,60261.0,Emirates Stadium,Jarred Gillett,3.0,51.5550,-0.1086,2024-08-17T14:00:00,21.734999,4,0,0,2024-08-17_Arsenal_Wolves,WOL,ARS,1.628300,0.575835,True,True,https://understat.com/match/26604,0.0,0.5796,0.575835,-1.052465,10.818182,2.0,3.0,2.1894,1.628300,1.052465,7.769231,14.0,3,1.0,2,5,Arsenal,Wolves,2024-08-17_Arsenal_Wolves,Arsenal,Wolves,1.18,7.50,13.00,2,5,2.0,5.0
3,3,497413,Everton,Brighton,0.0,3.0,AWAY_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,0.5,1.4,39217.0,Goodison Park,Simon Hooper,4.0,53.4389,-2.9664,2024-08-17T14:00:00,17.587500,0,0,0,2024-08-17_Everton_Brighton,BRI,EVE,0.405325,1.790830,True,True,https://understat.com/match/26605,3.0,2.5287,1.790830,1.385505,7.916667,5.0,0.0,0.3078,0.405325,-1.385505,18.333333,4.0,4,1.0,2,2,Everton,Brighton,2024-08-17_Everton_Brighton,Everton,Brighton,2.63,3.30,2.63,2,2,2.0,2.0
4,4,497414,Newcastle Utd,Southampton,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,0.3,1.8,52196.0,St James' Park,Craig Pawson,5.0,54.9756,-1.6217,2024-08-17T14:00:00,18.622499,0,0,0,2024-08-17_Newcastle Utd_Southampton,SOU,NEW,0.433489,1.954830,True,True,https://understat.com/match/26606,0.0,2.5586,1.954830,1.521341,3.789474,13.0,3.0,0.2774,0.433489,-1.521341,16.250000,4.0,5,1.0,2,4,Newcastle Utd,Southampton,2024-08-17_Newcastle Utd_Southampton,Newcastle Utd,Southampton,1.36,5.25,8.00,2,4,2.0,4.0


In [1556]:
table_df = table_df.drop(columns=['team_1', 'team_2','Team1', 'Team2'])

In [1557]:
table_df.to_csv("fact_table.csv")

In [1558]:
df = table_df

In [1559]:
df.columns

Index(['Unnamed: 0.1', 'Match ID', 'Team 1', 'Team 2', 'Team 1 Score',
       'Team 2 Score', 'Winner', 'Prev Team 1 Played', 'Prev Team 1 Won',
       'Prev Team 1 Drawn', 'Prev Team 1 Lost', 'Prev Team 1 Goals Scored',
       'Prev Team 1 Goals Conceded', 'Prev Team 1 Goal Difference',
       'Prev Team 1 Points', 'Prev Team 1 Form', 'Prev Team 2 Played',
       'Prev Team 2 Won', 'Prev Team 2 Drawn', 'Prev Team 2 Lost',
       'Prev Team 2 Goals Scored', 'Prev Team 2 Goals Conceded',
       'Prev Team 2 Goal Difference', 'Prev Team 2 Points', 'Prev Team 2 Form',
       'Date', 'Time', 'Wk', 'Day', 'xG', 'xG.1', 'Attendance', 'Venue',
       'Referee', 'StadiumID', 'Latitude', 'Longitude', 'Datetime',
       'Temperature', 'H2H_Team_1_Wins_Last_4', 'H2H_Team_2_Wins_Last_4',
       'H2H_Draws_Last_4', 'Composite Key', 'team_2_code', 'team_1_code',
       'team_1_xg', 'team_2_xg', 'is_result', 'has_data', 'url',
       'team2_points', 'team2_expected_points', 'team2_np_xg',
       'tea

### FE: Average ppda/deep completion

In [1560]:
def calculate_weighted_ppda_deep_completions(df):
    # Initialize dictionaries to track cumulative sums and games played
    cumulative_ppda = {}
    cumulative_deep = {}
    cumulative_fdr = {}
    games_played = {}

    # Create new columns for weighted average PPDA and deep completions
    cum_ppda_team1 = []
    cum_ppda_team2 = []
    cum_deep_team1 = []
    cum_deep_team2 = []
    avg_ppda_team1 = []
    avg_ppda_team2 = []
    avg_deep_team1 = []
    avg_deep_team2 = []
    weighted_avg_ppda_team1 = []
    weighted_avg_ppda_team2 = []
    weighted_avg_deep_team1 = []
    weighted_avg_deep_team2 = []

    # Iterate over rows
    for index, row in df.iterrows():
        team1 = row["Team 1"]
        team2 = row["Team 2"]

        # Initialize if not present in the dictionary
        for team in (team1, team2):
            if team not in cumulative_ppda:
                cumulative_ppda[team] = 0
                cumulative_deep[team] = 0
                cumulative_fdr[team] = 0
                games_played[team] = 0

        # Record cumulative values for the current row before updating them
        cum_ppda_team1.append(cumulative_ppda[team1])
        cum_ppda_team2.append(cumulative_ppda[team2])
        cum_deep_team1.append(cumulative_deep[team1])
        cum_deep_team2.append(cumulative_deep[team2])

        # Calculate averages based on games played before this match
        if games_played[team1] > 0:
            avg_ppda_team1.append(cumulative_ppda[team1] / games_played[team1])
            avg_deep_team1.append(cumulative_deep[team1] / games_played[team1])
            weighted_avg_ppda_team1.append((cumulative_ppda[team1] / games_played[team1]) * (cumulative_fdr[team1] / games_played[team1]))
            weighted_avg_deep_team1.append((cumulative_deep[team1] / games_played[team1]) * (cumulative_fdr[team1] / games_played[team1]))
        else:
            avg_ppda_team1.append(0)
            avg_deep_team1.append(0)
            weighted_avg_ppda_team1.append(0)
            weighted_avg_deep_team1.append(0)

        if games_played[team2] > 0:
            avg_ppda_team2.append(cumulative_ppda[team2] / games_played[team2])
            avg_deep_team2.append(cumulative_deep[team2] / games_played[team2])
            weighted_avg_ppda_team2.append((cumulative_ppda[team2] / games_played[team2]) * (cumulative_fdr[team2] / games_played[team2]))
            weighted_avg_deep_team2.append((cumulative_deep[team2] / games_played[team2]) * (cumulative_fdr[team2] / games_played[team2]))
        else:
            avg_ppda_team2.append(0)
            avg_deep_team2.append(0)
            weighted_avg_ppda_team2.append(0)
            weighted_avg_deep_team2.append(0)

        # Update cumulative PPDA, deep completions, and FDR after recording the current match
        cumulative_ppda[team1] += row['team1_ppda']
        cumulative_ppda[team2] += row['team2_ppda']
        cumulative_deep[team1] += row['team1_deep_completions']
        cumulative_deep[team2] += row['team2_deep_completions']
        cumulative_fdr[team1] += row['team_1_fdr']
        cumulative_fdr[team2] += row['team_2_fdr']

        # Increment games played after using the counts
        games_played[team1] += 1
        games_played[team2] += 1

    # Add new columns to the DataFrame
    df['Cum PPDA Team 1'] = cum_ppda_team1
    df['Cum PPDA Team 2'] = cum_ppda_team2
    df['Avg PPDA Team 1'] = avg_ppda_team1
    df['Avg PPDA Team 2'] = avg_ppda_team2
    df['Cum Deep Completions Team 1'] = cum_deep_team1
    df['Cum Deep Completions Team 2'] = cum_deep_team2
    df['Avg Deep Completions Team 1'] = avg_deep_team1
    df['Avg Deep Completions Team 2'] = avg_deep_team2
    df['Weighted Avg PPDA Team 1'] = weighted_avg_ppda_team1
    df['Weighted Avg PPDA Team 2'] = weighted_avg_ppda_team2
    df['Weighted Avg Deep Completions Team 1'] = weighted_avg_deep_team1
    df['Weighted Avg Deep Completions Team 2'] = weighted_avg_deep_team2

    return df

# Example usage with your dataset
updated_data = calculate_weighted_ppda_deep_completions(df)

# Display selected rows with weighted cumulative values and averages
columns_to_display = [
    'Match ID', 'Team 1', 'Team 2', 'team1_ppda', 'team2_ppda',
    'Weighted Avg PPDA Team 1', 'Weighted Avg PPDA Team 2',
    'Weighted Avg Deep Completions Team 1', 'Weighted Avg Deep Completions Team 2'
]
display_df = updated_data[columns_to_display]


In [1561]:
display_df.head(12)

,Match ID,Team 1,Team 2,team1_ppda,team2_ppda,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2
0,497410,Manchester Utd,Fulham,7.379310,10.833333,0.000000,0.000000,0.0,0.0
1,497411,Ipswich Town,Liverpool,18.777778,8.739130,0.000000,0.000000,0.0,0.0
2,497412,Arsenal,Wolves,7.769231,10.818182,0.000000,0.000000,0.0,0.0
3,497413,Everton,Brighton,18.333333,7.916667,0.000000,0.000000,0.0,0.0
4,497414,Newcastle Utd,Southampton,16.250000,3.789474,0.000000,0.000000,0.0,0.0
5,497415,Nottingham Forest,Bournemouth,8.653846,9.954545,0.000000,0.000000,0.0,0.0
6,497416,West Ham,Aston Villa,9.333333,11.304348,0.000000,0.000000,0.0,0.0
7,497417,Brentford,Crystal Palace,12.666667,7.724138,0.000000,0.000000,0.0,0.0
8,497418,Chelsea,Manchester City,10.064516,15.058824,0.000000,0.000000,0.0,0.0
9,497419,Leicester City,Tottenham,20.687500,5.482759,0.000000,0.000000,0.0,0.0


In [1562]:
updated_data.head(25)

,Unnamed: 0.1,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Team_1_Wins_Last_4,H2H_Team_2_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,home_np_xg_difference,team1_ppda,team1_deep_completions,Unnamed: 0,gameweek,team_1_fdr,team_2_fdr,composite_key,B365H,B365D,B365A,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2
0,0,497410,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-16,19:00,1,Fri,2.4,0.4,73297.0,Old Trafford,Robert Jones,1.0,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,3,1,0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1,1.0,2,3,2024-08-16_Manchester Utd_Fulham,1.60,4.20,5.25,2,3,2.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.00
1,1,497411,Ipswich Town,Liverpool,0.0,2.0,AWAY_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,11:30,1,Sat,0.5,2.6,30014.0,Portman Road Stadium,Tim Robinson,2.0,52.0567,1.1450,2024-08-17T11:30:00,22.212500,0,0,0,2024-08-17_Ipswich Town_Liverpool,LIV,IPS,0.342601,3.929060,True,True,https://understat.com/match/26603,3.0,2.9562,3.929060,3.586459,8.739130,13.0,0.0,0.0258,0.342601,-3.586459,18.777778,2.0,2,1.0,5,2,2024-08-17_Ipswich Town_Liverpool,8.50,5.50,1.33,5,2,5.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.00
2,2,497412,Arsenal,Wolves,2.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,1.2,0.5,60261.0,Emirates Stadium,Jarred Gillett,3.0,51.5550,-0.1086,2024-08-17T14:00:00,21.734999,4,0,0,2024-08-17_Arsenal_Wolves,WOL,ARS,1.628300,0.575835,True,True,https://understat.com/match/26604,0.0,0.5796,0.575835,-1.052465,10.818182,2.0,3.0,2.1894,1.628300,1.052465,7.769231,14.0,3,1.0,2,5,2024-08-17_Arsenal_Wolves,1.18,7.50,13.00,2,5,2.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.00
3,3,497413,Everton,Brighton,0.0,3.0,AWAY_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,0.5,1.4,39217.0,Goodison Park,Simon Hooper,4.0,53.4389,-2.9664,2024-08-17T14:00:00,17.587500,0,0,0,2024-08-17_Everton_Brighton,BRI,EVE,0.405325,1.790830,True,True,https://understat.com/match/26605,3.0,2.5287,1.790830,1.385505,7.916667,5.0,0.0,0.3078,0.405325,-1.385505,18.333333,4.0,4,1.0,2,2,2024-08-17_Everton_Brighton,2.63,3.30,2.63,2,2,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.00
4,4,497414,Newcastle Utd,Southampton,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,0.3,1.8,52196.0,St James' Park,Craig Pawson,5.0,54.9756,-1.6217,2024-08-17T14:00:00,18.622499,0,0,0,2024-08-17_Newcastle Utd_Southampton,SOU,NEW,0.433489,1.954830,True,True,https://understat.com/match/26606,0.0,2.5586,1.954830,1.521341,3.789474,13.0,3.0,0.2774,0.433489,-1.521341,16.250000,4.0,5,1.0,2,4,2024-08-17_Newcastle Ut

In [1563]:
updated_data.columns

/opt/anaconda3/lib/python3.11/site-packages/IPython/core/displayhook.py:281: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


Index(['Unnamed: 0.1', 'Match ID', 'Team 1', 'Team 2', 'Team 1 Score',
       'Team 2 Score', 'Winner', 'Prev Team 1 Played', 'Prev Team 1 Won',
       'Prev Team 1 Drawn', 'Prev Team 1 Lost', 'Prev Team 1 Goals Scored',
       'Prev Team 1 Goals Conceded', 'Prev Team 1 Goal Difference',
       'Prev Team 1 Points', 'Prev Team 1 Form', 'Prev Team 2 Played',
       'Prev Team 2 Won', 'Prev Team 2 Drawn', 'Prev Team 2 Lost',
       'Prev Team 2 Goals Scored', 'Prev Team 2 Goals Conceded',
       'Prev Team 2 Goal Difference', 'Prev Team 2 Points', 'Prev Team 2 Form',
       'Date', 'Time', 'Wk', 'Day', 'xG', 'xG.1', 'Attendance', 'Venue',
       'Referee', 'StadiumID', 'Latitude', 'Longitude', 'Datetime',
       'Temperature', 'H2H_Team_1_Wins_Last_4', 'H2H_Team_2_Wins_Last_4',
       'H2H_Draws_Last_4', 'Composite Key', 'team_2_code', 'team_1_code',
       'team_1_xg', 'team_2_xg', 'is_result', 'has_data', 'url',
       'team2_points', 'team2_expected_points', 'team2_np_xg',
       'tea

In [1564]:
updated_data.to_csv("updated_data.csv")

In [1565]:
df = updated_data
df.head(12)

,Unnamed: 0.1,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Team_1_Wins_Last_4,H2H_Team_2_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,home_np_xg_difference,team1_ppda,team1_deep_completions,Unnamed: 0,gameweek,team_1_fdr,team_2_fdr,composite_key,B365H,B365D,B365A,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2
0,0,497410,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-16,19:00,1,Fri,2.4,0.4,73297.0,Old Trafford,Robert Jones,1.0,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,3,1,0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1,1.0,2,3,2024-08-16_Manchester Utd_Fulham,1.60,4.20,5.25,2,3,2.0,3.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
1,1,497411,Ipswich Town,Liverpool,0.0,2.0,AWAY_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,11:30,1,Sat,0.5,2.6,30014.0,Portman Road Stadium,Tim Robinson,2.0,52.0567,1.1450,2024-08-17T11:30:00,22.212500,0,0,0,2024-08-17_Ipswich Town_Liverpool,LIV,IPS,0.342601,3.929060,True,True,https://understat.com/match/26603,3.0,2.9562,3.929060,3.586459,8.739130,13.0,0.0,0.0258,0.342601,-3.586459,18.777778,2.0,2,1.0,5,2,2024-08-17_Ipswich Town_Liverpool,8.50,5.50,1.33,5,2,5.0,2.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2,2,497412,Arsenal,Wolves,2.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,1.2,0.5,60261.0,Emirates Stadium,Jarred Gillett,3.0,51.5550,-0.1086,2024-08-17T14:00:00,21.734999,4,0,0,2024-08-17_Arsenal_Wolves,WOL,ARS,1.628300,0.575835,True,True,https://understat.com/match/26604,0.0,0.5796,0.575835,-1.052465,10.818182,2.0,3.0,2.1894,1.628300,1.052465,7.769231,14.0,3,1.0,2,5,2024-08-17_Arsenal_Wolves,1.18,7.50,13.00,2,5,2.0,5.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
3,3,497413,Everton,Brighton,0.0,3.0,AWAY_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,0.5,1.4,39217.0,Goodison Park,Simon Hooper,4.0,53.4389,-2.9664,2024-08-17T14:00:00,17.587500,0,0,0,2024-08-17_Everton_Brighton,BRI,EVE,0.405325,1.790830,True,True,https://understat.com/match/26605,3.0,2.5287,1.790830,1.385505,7.916667,5.0,0.0,0.3078,0.405325,-1.385505,18.333333,4.0,4,1.0,2,2,2024-08-17_Everton_Brighton,2.63,3.30,2.63,2,2,2.0,2.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
4,4,497414,Newcastle Utd,Southampton,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,0.3,1.8,52196.0,St James' Park,Craig Pawson,5.0,54.9756,-1.6217,2024-08-17T14:00:00,18.622499,0,0,0,2024-08-17_Newcastle Utd_Southampton,SOU,NEW,0.433489,1.954830,True,True,https://understat.com/match/26606,0.0,2.5586,1.954830,1.521341,3.789474,13.0,3.0,0.2774,0.433489,-1.521341,16.250000,4.0,5,1.0,2,4,2024-08-17_Newcastle Utd_Southampton,1.36,5.25,8.00,2,4,2.0,4.0,0.00000

### FE Average win/draw/points/goalscored ... stats

In [1619]:
import pandas as pd

def calculate_cumulative_stats(dataframe):
    # Initialize dictionaries to track cumulative stats and games played for each team
    cumulative_stats = {}
    games_played = {}

    # List of stats to track
    stats_columns = [
        'Won', 'Drawn', 'Lost', 'Goals Scored', 'Goals Conceded', 
        'Goal Difference', 'Points'
    ]

    # Initialize columns in the DataFrame to store cumulative and average stats for each team
    for stat in stats_columns:
        dataframe[f'Cum Team 1 {stat}'] = 0
        dataframe[f'Cum Team 2 {stat}'] = 0
        dataframe[f'Avg Team 1 {stat}'] = 0.0
        dataframe[f'Avg Team 2 {stat}'] = 0.0

    # Lists to store the values temporarily
    temp_cum_team1 = {stat: [] for stat in stats_columns}
    temp_cum_team2 = {stat: [] for stat in stats_columns}
    temp_avg_team1 = {stat: [] for stat in stats_columns}
    temp_avg_team2 = {stat: [] for stat in stats_columns}

    # Iterate over rows
    for index, row in dataframe.iterrows():
        team1 = row['Team 1']
        team2 = row['Team 2']

        if team1 not in cumulative_stats:
            cumulative_stats[team1] = {stat: 0 for stat in stats_columns}
            games_played[team1] = 0
        if team2 not in cumulative_stats:
            cumulative_stats[team2] = {stat: 0 for stat in stats_columns}
            games_played[team2] = 0

        for stat in stats_columns:
            # Update cumulative stats
            cumulative_stats[team1][stat] += row[f'Prev Team 1 {stat}']
            cumulative_stats[team2][stat] += row[f'Prev Team 2 {stat}']

        # Calculate averages using the count of games played so far
        for stat in stats_columns:
            if games_played[team1] > 0:
                avg_team_1_stat = cumulative_stats[team1][stat] / games_played[team1]
                temp_avg_team1[stat].append(avg_team_1_stat)
                temp_cum_team1[stat].append(cumulative_stats[team1][stat])
            else:
                temp_avg_team1[stat].append(0)
                temp_cum_team1[stat].append(0)
            
            if games_played[team2] > 0:
                avg_team_2_stat = cumulative_stats[team2][stat] / games_played[team2]
                temp_avg_team2[stat].append(avg_team_2_stat)
                temp_cum_team2[stat].append(cumulative_stats[team2][stat])
            else:
                temp_avg_team2[stat].append(0)
                temp_cum_team2[stat].append(0)

        # Increment games played after calculation
        games_played[team1] += 1
        games_played[team2] += 1

    # Assign lists to DataFrame columns
    for stat in stats_columns:
        dataframe[f'Cum Team 1 {stat}'] = temp_cum_team1[stat]
        dataframe[f'Cum Team 2 {stat}'] = temp_cum_team2[stat]
        dataframe[f'Avg Team 1 {stat}'] = temp_avg_team1[stat]
        dataframe[f'Avg Team 2 {stat}'] = temp_avg_team2[stat]

    return dataframe

# Assuming 'data' is your DataFrame loaded from 'your_data.csv'
updated_data = calculate_cumulative_stats(updated_data)

# Display selected rows with cumulative values and averages
columns_to_display = ['Match ID', 'Team 1', 'Team 2'] + [col for col in data.columns if 'Avg' in col or 'Cum' in col]
updated_data[columns_to_display].head()


,Match ID,Team 1,Team 2,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2,Cum Team 1 Won,Cum Team 2 Won,Avg Team 1 Won,Avg Team 2 Won,Cum Team 1 Drawn,Cum Team 2 Drawn,Avg Team 1 Drawn,Avg Team 2 Drawn,Cum Team 1 Lost,Cum Team 2 Lost,Avg Team 1 Lost,Avg Team 2 Lost,Cum Team 1 Goals Scored,Cum Team 2 Goals Scored,Avg Team 1 Goals Scored,Avg Team 2 Goals Scored,Cum Team 1 Goals Conceded,Cum Team 2 Goals Conceded,Avg Team 1 Goals Conceded,Avg Team 2 Goals Conceded,Cum Team 1 Goal Difference,Cum Team 2 Goal Difference,Avg Team 1 Goal Difference,Avg Team 2 Goal Difference,Cum Team 1 Points,Cum Team 2 Points,Avg Team 1 Points,Avg Team 2 Points
0,497410,Manchester Utd,Fulham,2,3,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0
1,497411,Ipswich Town,Liverpool,5,2,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0
2,497412,Arsenal,Wolves,2,5,2.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0
3,497413,Everton,Brighton,2,2,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0
4,497414,Newcastle Utd,Southampton,2,4,2.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0


In [1621]:
# Displaying rows 20 to 30 from the DataFrame
updated_data.iloc[29:31]


,Unnamed: 0.1,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Team_1_Wins_Last_4,H2H_Team_2_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,home_np_xg_difference,team1_ppda,team1_deep_completions,Unnamed: 0,gameweek,team_1_fdr,team_2_fdr,composite_key,B365H,B365D,B365A,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2,Cum Team 1 Won,Cum Team 2 Won,Avg Team 1 Won,Avg Team 2 Won,Cum Team 1 Drawn,Cum Team 2 Drawn,Avg Team 1 Drawn,Avg Team 2 Drawn,Cum Team 1 Lost,Cum Team 2 Lost,Avg Team 1 Lost,Avg Team 2 Lost,Cum Team 1 Goals Scored,Cum Team 2 Goals Scored,Avg Team 1 Goals Scored,Avg Team 2 Goals Scored,Cum Team 1 Goals Conceded,Cum Team 2 Goals Conceded,Avg Team 1 Goals Conceded,Avg Team 2 Goals Conceded,Cum Team 1 Goal Difference,Cum Team 2 Goal Difference,Avg Team 1 Goal Difference,Avg Team 2 Goal Difference,Cum Team 1 Points,Cum Team 2 Points,Avg Team 1 Points,Avg Team 2 Points
29,29,497436,Manchester Utd,Liverpool,0.0,3.0,AWAY_TEAM,2,1,0,1,2,2,0,3,1.5,2,2,0,0,4,0,4,6,3.0,2024-09-01,15:00,3,Sun,1.4,1.8,73738.0,Old Trafford,Anthony Taylor,1.0,53.4631,-2.2914,2024-09-01T15:00:00,19.213001,0,1,3,2024-09-01_Manchester Utd_Liverpool,LIV,MUN,1.50087,2.10397,True,True,https://understat.com/match/26631,3.0,1.7849,2.10397,0.603100,11.692308,9.0,0.0,0.9689,1.500870,-0.603100,12.500000,6.0,30,3.0,5,3,2024-09-01_Manchester Utd_Liverpool,3.7,4.2,1.85,10,7,3.333333,2.333333,24.157088,18.869565,12.078544,9.434783,15.0,34.0,7.500000,17.0,30.19636,18.869565,18.750000,34.000000,2,3,1.0,1.5,0,0,0.0,0.0,1,0,0.5,0.0,3,6,1.500000,3.000000,2,0,1.000000,0.000000,1,6,0.500000,3.000000,6,9,3.0,4.5
30,30,497447,Southampton,Manchester Utd,0.0,3.0,AWAY_TEAM,3,0,0,3,1,5,-4,0,0.0,3,1,0,2,2,5,-3,3,1.0,2024-09-14,11:30,4,Sat,1.1,2.6,31144.0,St Mary's Stadium,Stuart Attwell,15.0,50.9058,-1.3913,2024-09-14T11:30:00,16.136499,0,1,3,2024-09-14_Southampton_Manchester Utd,MUN,SOU,1.35593,3.02693,True,True,https://understat.com/match/26632,3.0,2.5032,3.02693,2.481613,10.344828,17.0,0.0,0.3435,0.545317,-2.481613,10.857143,3.0,31,4.0,3,2,2024-09-14_Southampton_Manchester Utd,4.5,4.0,1.70,13,12,3.250000,3.000000,20.149722,36.657088,6.716574,12.219029,31.0,21.0,10.333333,7.0,22.38858,40.730098,34.444444,23.333333,0,3,0.0,1.0,0,0,0.0,0.0,6,3,2.0,1.0,1,5,0.333333,1.666667,8,7,2.666667,2.333333,-7,-2,-2.333333,-0.666667,0,9,0.0,3.0


In [1625]:
updated_data.to_csv("fact_table.csv")